TOPIC: Data Warehousing Fundamentals
   1. Design a data warehouse schema for a retail company that includes dimension tables for products, customers, and time. Implement the schema using a relational database management system (RDBMS) of your choice.
   2. Create a fact table that captures sales data, including product ID, customer ID, date, and sales amount. Populate the fact table with sample data.
   3. Write SQL queries to retrieve sales data from the data warehouse, including aggregations and filtering based on different dimensions.



1.To design a data warehouse schema for a retail company, we will create dimension tables for products, customers, and time. We will also create a fact table to capture sales data. For this example, we will use a relational database management system (RDBMS) and assume the following table structures:

Product Dimension Table:

ProductID (Primary Key)
ProductName
Category
Price
...
Customer Dimension Table:

CustomerID (Primary Key)
FirstName
LastName
Address
...
Time Dimension Table:

DateKey (Primary Key)
Date
Year
Month
Day
...
Sales Fact Table:

SaleID (Primary Key)
ProductID (Foreign Key to Product Dimension Table)
CustomerID (Foreign Key to Customer Dimension Table)
DateKey (Foreign Key to Time Dimension Table)
SalesAmount

2.Product Dimension Table:

ProductID	ProductName	Category	Price
1	Laptop	Electronics	1000
2	Smartphone	Electronics	800
3	T-Shirt	Clothing	20
4	Shoes	Clothing	50
...	...	...	...
Customer Dimension Table:

CustomerID	FirstName	LastName	Address
1	John	Doe	123 Main St.
2	Jane	Smith	456 Elm St.
3	David	Johnson	789 Oak St.
4	Emily	Davis	987 Pine St.
...	...	...	...
Time Dimension Table:

DateKey	Date	Year	Month	Day
1	2023-01-01	2023	01	01
2	2023-01-02	2023	01	02
3	2023-01-03	2023	01	03
4	2023-01-04	2023	01	04
...	...	...	...	...
Now, let's populate the fact table with sample sales data:

Sales Fact Table:

SaleID	ProductID	CustomerID	DateKey	SalesAmount
1	1	1	1	1500
2	2	2	1	800
3	3	3	2	60
4	4	4	2	100
...	...	...	...	...

In [ ]:
SELECT
    p.ProductName,
    SUM(f.SalesAmount) AS TotalSalesAmount
FROM
    SalesFactTable f
    INNER JOIN ProductDimensionTable p ON f.ProductID = p.ProductID
GROUP BY
    p.ProductName;

SELECT
    c.FirstName,
    c.LastName,
    SUM(f.SalesAmount) AS TotalSalesAmount
FROM
    SalesFactTable f
    INNER JOIN CustomerDimensionTable c ON f.CustomerID = c.CustomerID
    INNER JOIN TimeDimensionTable t ON f.DateKey = t.DateKey
WHERE
    t.Year = 2023
GROUP BY
    c.FirstName,
    c.LastName;

SELECT
    t.Year,
    t.Month,
    SUM(f.SalesAmount) AS TotalSalesAmount
FROM
    SalesFactTable f
    INNER JOIN TimeDimensionTable t ON f.DateKey = t.DateKey
GROUP BY
    t.Year,
    t.Month;


TOPIC: ETL and Data Integration
  1. Design an ETL process using a programming language (e.g., Python) to extract data from a source system (e.g., CSV files), transform it by applying certain business rules or calculations, and load it into a data warehouse.
  2. Implement the ETL process by writing code that performs the extraction, transformation, and loading steps.




In [ ]:
import csv
import sqlite3

def extract_data(file_path):
    with open(file_path, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        header = next(csv_reader)
        data = [row for row in csv_reader]
    return data

def transform_data(data):
    transformed_data = []
    for row in data:
        product_id = int(row[0])
        sales_amount = float(row[1])
        transformed_data.append((product_id, sales_amount))
    return transformed_data

def load_data(data):
    conn = sqlite3.connect('data_warehouse.db')
    cursor = conn.cursor()
    for row in data:
        product_id, sales_amount = row
        cursor.execute("INSERT INTO SalesFactTable (ProductID, SalesAmount) VALUES (?, ?)",
                       (product_id, sales_amount))
    conn.commit()
    conn.close()

file_path = 'sales_data.csv'
extracted_data = extract_data(file_path)
transformed_data = transform_data(extracted_data)
load_data(transformed_data)


TOPIC: Dimensional Modeling and Schemas
   1. Design a star schema for a university database, including a fact table for student enrollments and dimension tables for students, courses, and time. Implement the schema using a database of your choice.
   2. Write SQL queries to retrieve data from the star schema, including aggregations and joins between the fact table and dimension tables.


To design a star schema for a university database, we will create a fact table for student enrollments and dimension tables for students, courses, and time. For this example, let's assume the following table structures:

Student Dimension Table:

StudentID (Primary Key)
FirstName
LastName
Age
Gender
...
Course Dimension Table:

CourseID (Primary Key)
CourseName
Department
Professor
...
Time Dimension Table:

DateKey (Primary Key)
Date
Year
Month
Day
...
Enrollment Fact Table:

EnrollmentID (Primary Key)
StudentID (Foreign Key to Student Dimension Table)
CourseID (Foreign Key to Course Dimension Table)
DateKey (Foreign Key to Time Dimension Table)
Grade

In [ ]:
SELECT
    c.CourseName,
    COUNT(e.EnrollmentID) AS TotalEnrollments
FROM
    EnrollmentFactTable e
    INNER JOIN CourseDimensionTable c ON e.CourseID = c.CourseID
GROUP BY
    c.CourseName;

SELECT
    c.CourseName,
    SUM(CASE WHEN s.Gender = 'M' THEN 1 ELSE 0 END) AS MaleStudents,
    SUM(CASE WHEN s.Gender = 'F' THEN 1 ELSE 0 END) AS FemaleStudents
FROM
    EnrollmentFactTable e
    INNER JOIN CourseDimensionTable c ON e.CourseID = c.CourseID
    INNER JOIN StudentDimensionTable s ON e.StudentID = s.StudentID
GROUP BY
    c.CourseName;

SELECT
    c.CourseName,
    AVG(e.Grade) AS AverageGrade
FROM
    EnrollmentFactTable e
    INNER JOIN CourseDimensionTable c ON e.CourseID = c.CourseID
    INNER JOIN TimeDimensionTable t ON e.DateKey = t.DateKey
WHERE
    t.Year = 2023
GROUP BY
    c.CourseName;


TOPIC: Performance Optimization and Querying
    1. Scenario: You need to improve the performance of your data loading process in the data warehouse. Write a Python script that implements the following optimizations:
Utilize batch processing techniques to load data in bulk instead of individual row insertion.
      b)  Implement multi-threading or multiprocessing to parallelize the data loading process.
      c)  Measure the time taken to load a specific amount of data before and after implementing these optimizations.


In [ ]:
import csv
import psycopg2
from multiprocessing import Pool
from datetime import datetime

file_path = 'data.csv'
batch_size = 1000
num_processes = 4

def load_data(connection, cursor, data):
    query = "COPY SalesFactTable (ProductID, SalesAmount) FROM STDIN WITH CSV"
    cursor.copy_expert(query, data)
    connection.commit()

def process_batch(batch):
    connection = psycopg2.connect(
        host='your_host',
        port='your_port',
        database='your_database',
        user='your_username',
        password='your_password'
    )
    cursor = connection.cursor()

    try:
        load_data(connection, cursor, batch)
    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Error: {error}")
    finally:
        cursor.close()
        connection.close()

def load_data_optimized():
    start_time = datetime.now()

    with open(file_path, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        header = next(csv_reader)

        data_batches = [list(csv_reader[i:i + batch_size]) for i in range(0, len(csv_reader), batch_size)]

        pool = Pool(processes=num_processes)
        pool.map(process_batch, data_batches)
        pool.close()
        pool.join()

    end_time = datetime.now()
    loading_time = end_time - start_time
    print(f"Data loading completed in: {loading_time}")

print("Loading time without optimizations:")
load_data_optimized()

print("Loading time with optimizations:")
load_data_optimized()
